In [1]:
import datetime as dt
import itertools
from pathlib import Path

import dask
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
from scipy import interpolate
import xarray as xr

from mcs_prime import McsTracks, PATHS
from mcs_prime.util import update_progress
%matplotlib inline

/home/users/mmuetz/miniconda3/envs/mcs_prime_env/lib/python3.10/site-packages/pyproj/__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


Load all tracks from 2000/6-2020/12

In [3]:
tracks = McsTracks.mfopen(str(PATHS['statsdir'] / 'mcs_tracks_final_extc_*.0000.nc'), 
                          PATHS['pixeldir'])
ntracks = len(tracks.dstracks.tracks)
tracks

McsTracks[2000-06-01 00:30:00, 2020-12-31 22:30:00, ntracks=696293]

`McsTracks` provides a wrapper around a `dstracks` object, which is an `xarray.DataSet`.

In [4]:
tracks.dstracks
# How does dstracks look so good? It implements _repr_html_() which returns all the HTML you can see below.
# Looks like a lot of work for this, but a simplified version could be nice for other objects.

<xarray.Dataset>
Dimensions:                  (tracks: 696293, times: 400, mergers: 50, nmaxpf: 3)
Coordinates:
  * tracks                   (tracks) int64 0 1 2 3 ... 696290 696291 696292
  * times                    (times) int64 0 1 2 3 4 5 ... 395 396 397 398 399
  * mergers                  (mergers) int64 0 1 2 3 4 5 6 ... 44 45 46 47 48 49
  * nmaxpf                   (nmaxpf) int64 0 1 2
Data variables: (12/55)
    track_duration           (tracks) int32 dask.array<chunksize=(21142,), meta=np.ndarray>
    start_status             (tracks) int32 dask.array<chunksize=(21142,), meta=np.ndarray>
    end_status               (tracks) int32 dask.array<chunksize=(21142,), meta=np.ndarray>
    start_basetime           (tracks) datetime64[ns] 2000-06-01T00:30:00 ... ...
    end_basetime             (tracks) datetime64[ns] 2000-06-01T12:30:00 ... ...
    start_split_cloudnumber  (tracks) int32 dask.array<chunksize=(21142,), meta=np.ndarray>
    ...                       ...
    pf_mcsstatus             (tracks, times) int64 dask.array<chunksize=(21142, 400), meta=np.ndarray>
    movement_distance        (tracks, times) float32 dask.array<chunksize=(21142, 400), meta=np.ndarray>
    movement_speed           (tracks, times) float32 dask.array<chunksize=(21142, 400), meta=np.ndarray>
    movement_theta           (tracks, times) float32 dask.array<chunksize=(21142, 400), meta=np.ndarray>
    movement_distance_x      (tracks, times) float32 dask.array<chunksize=(21142, 400), meta=np.ndarray>
    movement_distance_y      (tracks, times) float32 dask.array<chunksize=(21142, 400), meta=np.ndarray>
Attributes: (12/23)
    Title:                       Statistics of each MCS track
    Institution:                 Pacific Northwest National Laboratory
    Contact:                     Zhe Feng, zhe.feng@pnnl.gov
    Created_on:                  Wed Mar  9 18:38:34 2022
    startdate:                   20000601.0000
    enddate:                     20010101.0000
    ...                          ...
    PF_PF_min_majoraxis_thresh:  20
    coefs_pf_area:               [2874.05    89.825]
    coefs_pf_rr:                 [3.01657   0.0144461]
    coefs_pf_skew:               [0.194462  0.0100072]
    coefs_pf_heavyratio:         [3.419024 0.438709]
    max_speed_thresh:            50

In [9]:
np.nanmin(tracks.dstracks.ccs_area.values)

800.0

In [14]:
mcs_mask = (tracks.dstracks.mcs_status == 1).values

In [15]:
mcs_mask

array([[ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       ...,
       [False,  True,  True, ..., False, False, False],
       [False,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False]])

In [13]:
tracks.dstracks.ccs_area.load()

<xarray.DataArray 'ccs_area' (tracks: 696293, times: 400)>
array([[1602700.,  795100., 1220400., ...,      nan,      nan,      nan],
       [1114700., 1015400.,  604600., ...,      nan,      nan,      nan],
       [ 972600., 1001600.,  491200., ...,      nan,      nan,      nan],
       ...,
       [  24300.,  139600.,  211800., ...,      nan,      nan,      nan],
       [  31400.,  118400.,  189600., ...,      nan,      nan,      nan],
       [  72500.,  161500.,  174400., ...,      nan,      nan,      nan]],
      dtype=float32)
Coordinates:
  * tracks   (tracks) int64 0 1 2 3 4 5 ... 696288 696289 696290 696291 696292
  * times    (times) int64 0 1 2 3 4 5 6 7 8 ... 392 393 394 395 396 397 398 399
Attributes:
    _FillValue:  nan
    long_name:   Area of cold cloud shield
    units:       km^2

In [17]:
tracks.dstracks.ccs_area.values[mcs_mask].min()

40100.0

In [19]:
pf_mcs_mask = (tracks.dstracks.pf_mcsstatus == 1).values

In [20]:
tracks.dstracks.ccs_area.values[pf_mcs_mask].min()

800.0

In [22]:
tracks.dstracks.pf_rainrate.load()

<xarray.DataArray 'pf_rainrate' (tracks: 696293, times: 400, nmaxpf: 3)>
array([[[4.3586745, 4.3949337, 3.6471128],
        [4.139043 , 6.6412935, 7.8243256],
        [7.673871 , 4.785524 , 4.310433 ],
        ...,
        [      nan,       nan,       nan],
        [      nan,       nan,       nan],
        [      nan,       nan,       nan]],

       [[6.2787595, 5.8270364, 6.2364173],
        [5.355205 , 6.229534 , 5.654293 ],
        [5.0744576, 6.228387 , 4.8488407],
        ...,
        [      nan,       nan,       nan],
        [      nan,       nan,       nan],
        [      nan,       nan,       nan]],

       [[5.793713 , 4.039628 , 3.511933 ],
        [6.0371633, 4.3736053, 4.7054534],
        [5.3470216, 4.8912096, 3.5493836],
        ...,
...
        ...,
        [      nan,       nan,       nan],
        [      nan,       nan,       nan],
        [      nan,       nan,       nan]],

       [[4.480781 , 4.8903813, 3.0549686],
        [4.3955503, 3.6951964, 4.772977 ],
        [3.229958 , 3.6412544, 3.4674053],
        ...,
        [      nan,       nan,       nan],
        [      nan,       nan,       nan],
        [      nan,       nan,       nan]],

       [[3.4532275, 8.522544 , 4.405121 ],
        [5.940634 , 7.274376 , 2.522352 ],
        [9.05372  , 3.4765062,       nan],
        ...,
        [      nan,       nan,       nan],
        [      nan,       nan,       nan],
        [      nan,       nan,       nan]]], dtype=float32)
Coordinates:
  * tracks   (tracks) int64 0 1 2 3 4 5 ... 696288 696289 696290 696291 696292
  * times    (times) int64 0 1 2 3 4 5 6 7 8 ... 392 393 394 395 396 397 398 399
  * nmaxpf   (nmaxpf) int64 0 1 2
Attributes:
    _FillValue:  nan
    long_name:   Mean rain rate of PF
    units:       mm/h

In [26]:
np.nanmin(tracks.dstracks.pf_rainrate.values[pf_mcs_mask])

2.0092847